# 贪婪和恐慌指数分析

## 准备数据

读取恐慌贪婪指数，BTC价格数据，然后合并和清洗

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [31]:
# 恐慌和贪婪指数
fgi = pd.read_csv("../data/fear_greed_index.csv", index_col=0, parse_dates=True)

# BTC历史价格，binance, 现货
btc = pd.read_csv("../data/binance_btcusdt_1d.csv", index_col=0, parse_dates=True)

# 合并数据
fgi["btc_close"] = btc["close"]
fgi.dropna(inplace=True)

fgi.tail()

,value,classification,btc_close
date,,,
2024-09-17,33.0,Fear,60313.99
2024-09-18,45.0,Fear,61759.99
2024-09-19,49.0,Neutral,62947.99
2024-09-20,54.0,Neutral,63201.05
2024-09-21,54.0,Neutral,63040.01


## 可视化指数

In [32]:
# 添加移动平均
fgi["smooth"] = fgi["value"].rolling(7).mean()

fig = px.line(
    fgi,
    x=fgi.index,
    y=["value", "smooth"],
    labels={"value": "Fear and Greed Index", "smooth": "7-day MA"},
    title="Fear and Greed Index",
    width=800,
    height=500
)

fig.show()

## 持续天数

- 统计市场处于恐慌或极度恐慌的连续天数
- 统计市场处于贪婪或极度贪婪的连续天数
- 分析这两个指标与价格的关系

In [33]:
fear_days = np.zeros(len(fgi))
greed_days = np.zeros(len(fgi))

for i in range(1, len(fear_days)):
    if fgi["classification"].iloc[i] in ["Fear", "Extreme Fear"]:
        fear_days[i] = fear_days[i - 1] + 1
    if fgi["classification"].iloc[i] in ["Greed", "Extreme Greed"]:
        greed_days[i] = greed_days[i - 1] + 1

fgi["fear_days"] = fear_days
fgi["greed_days"] = greed_days

fgi.tail()

,value,classification,btc_close,smooth,fear_days,greed_days
date,,,,,,
2024-09-17,33.0,Fear,60313.99,39.000000,2.0,0.0
2024-09-18,45.0,Fear,61759.99,40.142857,3.0,0.0
2024-09-19,49.0,Neutral,62947.99,42.714286,0.0,0.0
2024-09-20,54.0,Neutral,63201.05,45.857143,0.0,0.0
2024-09-21,54.0,Neutral,63040.01,46.428571,0.0,0.0


In [35]:
# 创建分层图表，第一行是比特币价格，第二行是恐慌天数，第三行是贪婪天数
fig = make_subplots(
    rows=3,
    cols=1,
    subplot_titles=("Bitcoin price", "Cumulative days in fear", "Cumulative days in greed"),
    vertical_spacing=0.05,  # 通过这个值来调整子图之间的间距
)

# BTC价格
fig.add_trace(
    go.Scatter(x=fgi.index, y=fgi["btc_close"], mode="lines", showlegend=False),
    row=1,
    col=1,
)

# 恐慌天数
fig.add_trace(
    go.Scatter(x=fgi.index, y=fgi["fear_days"], mode="lines", showlegend=False),
    row=2,
    col=1,
)

# 贪婪天数
fig.add_trace(
    go.Scatter(x=fgi.index, y=fgi["greed_days"], mode="lines", showlegend=False),
    row=3,
    col=1,
)

# 更新图表布局
fig.update_xaxes(matches="x")  # 同步所有子图的x轴
fig.update_layout(
    height=1000,
    width=1200,
    title="Cumulative days in fear and greed",
)
fig.show()